In [ ]:
import pulp #線形計画計算ライブラリ
import pandas as pd #データフレーム作成
import numpy as np #計算用ライブラリ
import datetime #日付関係
import jpholiday #日本の祝日

#csvからデータフレーム作成
#自身の環境にあったパスを指定
file_path = 'ここにパスを入力'

#提出ファイルのデータフレーム
submit_df = pd.read_csv(file_path + 'submit.csv', index_col = 0, encoding = 'cp932')

#条件ファイルのデータフレーム
condition_df = pd.read_csv(file_path + 'condition.csv', index_col = 0, encoding = 'cp932')

#設定ファイルのデータフレーム
setting_df = pd.read_csv(file_path + 'setting.csv', index_col = 0, encoding = 'cp932')

#スタッフをリスト化
staff = submit_df.columns.tolist()

#時給と交通費を辞書リスト化
hourly_wage = {s:condition_df.loc[s,'時給'] for s in staff}
travelling_expenses = {s:condition_df.loc[s,'交通費'] for s in staff}

#日付をリスト化
day = submit_df.index.tolist()
#日付を数値リスト化
day2 = list(range(1,len(day)+1))

#勤務シフトをリスト化
#C[0]が休みになるように
C = setting_df.index.values #シフトパターン
C2 = setting_df.columns.values #カラム名
C3 = [] #シフトの勤務時間
C4 = [] #シフトパターン何人必要か（平日）
C5 = [] #シフトパターン何人必要か（平日）
for i in C:
    C3.append(setting_df.loc[i,'時間'])
    C4.append(setting_df.loc[i,'必要（平日）'])
    C5.append(setting_df.loc[i,'必要（祝祭日）'])
    
#スキル
#-1：要サポート、0：サポートスキルなし、1：サポートスキルあり
staff_skill = []
for s in staff:
    staff_skill.append(condition_df.loc[s,'スキル'])

staff_max = []
staff_min = []
staff_holiday = []
for s in staff:
    staff_max.append(condition_df.loc[s,'最大出勤数'])
    staff_min.append(condition_df.loc[s,'最小出勤数'])
    staff_holiday.append(condition_df.loc[s,'休日数'])
    
#問題の定義
prob = pulp.LpProblem('ShiftProblem', pulp.LpMinimize)

#変数の設定
#変数x[s,d,c]スタッフsがd日の勤務cであるかどうか
x = pulp.LpVariable.dicts('x',[(s,d,c) for s in staff for d in day for c in C], cat = 'Binary')

#目的関数の設定
#各スタッフの勤務時間×時給+勤務日数×交通費が最小になるよう設定
prob += pulp.lpSum([x[s,d,C[c]]*hourly_wage[s]*C3[c] + \
                    x[s,d,C[c]]*travelling_expenses[s] for s in staff for d in day for c in range(1,len(C))])
        
#その日の勤務は出勤パターンのどれか一つに必ず当てはまる
for s in staff:
    for d in day:
        prob += pulp.lpSum([x[s,d,c] for c in C]) == 1

#出勤日数を満たす（最大出勤数以下）
for s in staff:
    if np.isnan(condition_df.loc[s,'最大出勤数']) == False: #欠損値がない場合
        prob += pulp.lpSum([x[s,d,C[c]] for d in day for c in range(1,len(C))]) <= condition_df.loc[s,'最大出勤数']

#出勤日数を満たす（最小出勤数以上）
for s in staff:
    if np.isnan(condition_df.loc[s,'最小出勤数']) == False: #欠損値がない場合
        prob += pulp.lpSum([x[s,d,C[c]] for d in day for c in range(1,len(C))]) >= condition_df.loc[s,'最小出勤数']
        
#休みの日数を満たす
for s in staff:
    if np.isnan(condition_df.loc[s,'休日数']) == False: #欠損値がない場合
        prob += pulp.lpSum([x[s,d,C[0]] for d in day]) >= condition_df.loc[s,'休日数']
        
#希望シフトから、無関係なパターンを除外する
for s in staff:
    for d in day:
        buf = submit_df.loc[d,s] #スタッフsのd日の希望シフト
        if buf == C[0]: #休みのとき
            prob += x[s,d,C[0]] == 1 #休みで確定させる
        else:
            for c in range(1,len(C)):
                if setting_df.loc[C[c],buf] ==0: #希望シフトに収まらないシフトパターン
                    prob += x[s,d,C[c]] == 0 #除外する
        
#7日に1回以上は割り当てる
for s in staff:
    for d in day2[5:len(day2)-1]:
        prob += pulp.lpSum([x[s,day[d-h],C[c]] for h in range(1+6) for c in range(1,len(C))]) >= 1

#6連勤以上の禁止
for s in staff:
    for d in day2[4:len(day2)-1]:
        prob += pulp.lpSum([x[s,day[d-h],C[c]] for h in range(1+5) for c in range(1,len(C))]) <= 5

#任意の日、時間に必要なシフトパターンの人数を守る
for d in day:
    d2 = datetime.datetime.strptime(d, '%Y/%m/%d') #文字列→日付型
    if jpholiday.is_holiday(datetime.date(d2.year, d2.month, d2.day)) == True or d2.weekday() > 4:
        #祝祭日の場合
        for c in range(1,len(C)):
            prob += pulp.lpSum([x[s,d,C[c2]]*setting_df.loc[C[c],C[c2]] for s in staff for c2 in range(1,len(C))]) >= C5[c]
    else:
        #平日の場合
        for c in range(1,len(C)):
            prob += pulp.lpSum([x[s,d,C[c2]]*setting_df.loc[C[c],C[c2]] for s in staff for c2 in range(1,len(C))]) >= C4[c]

#要サポートとサポートできる人が同じ時間帯にいること/任意の日、時間についてのスキル合計が0以上
for d in day:
    for c1 in range(1,len(C)):
        pulp.lpSum([x[s,d,C[c2]]*setting_df.loc[C[c1],C[c2]]* \
                    condition_df.loc[s,'スキル'] for s in staff for c2 in range(1,len(C))]) >= 0
        
#結果の表示
status = prob.solve()
print('Status:',pulp.LpStatus[status])

In [ ]:
#出力用
output_df = pd.DataFrame(columns = staff, index = day)
output_df2 = pd.DataFrame(columns = staff, index = ['給与','交通費','合計'])
output_df3 = pd.concat([output_df,output_df2])

for s in staff:
    for d in day:
        for c in C:
            if x[s,d,c].value() == 1:
                output_df3.loc[d,s] = c
                
#給与など表示
for s in staff:
    output_df3.loc['給与',s] = 0
    output_df3.loc['交通費',s] = 0
    output_df3.loc['合計',s] = 0
    
for s in staff:
    for d in day:
        for c in range(1,len(C)):
            if x[s,d,C[c]].value() == 1:
                output_df3.loc['給与',s] += C3[c]*hourly_wage[s]
                output_df3.loc['交通費',s] += travelling_expenses[s]
                output_df3.loc['合計',s] += C3[c]*hourly_wage[s]+travelling_expenses[s]
                
#出力
output_df3.to_csv(file_path + 'result.csv', encoding = 'cp932')